# Estudio de funcionamiento de EDVR, herramienta de super resolución de vídeos
## TFG del Grado en Ingeniería Informática en la Universidad de Burgos. Curso 2020-21

Realizado por:
*  Gonzalo Murillo Montes 

Tutores:
*  Pedro Latorre Carmona
*  César Ignacio García Osorio

```bash
@misc{wang2020basicsr,
  author =       {Xintao Wang and Ke Yu and Kelvin C.K. Chan and
                  Chao Dong and Chen Change Loy},
  title =        {BasicSR},
  howpublished = {\url{https://github.com/xinntao/BasicSR}},
  year =         {2020}
}
```
Xintao Wang, Kelvin C.K. Chan, Ke Yu, Chao Dong, &amp; Chen Change Loy. (2019, May 7). EDVR: Video Restoration with Enhanced Deformable Convolutional Networks [PDF]. Ithaca, New York: Cornell University.

# **ÍNDICE**   
1. <a id="index"></a>[¿NTIRE19 CHALLENGE?](#id1)


2. <a id="index"></a>[¿QUÉ ES EDVR?](#id2)


3. <a id="index"></a>[PASOS PREVIOS](#id3)


4. <a id="index"></a>[DATASETS](#id4)

    4.1. <a id="index"></a>[REDS](#id5)
    
    4.2. <a id="index"></a>[ Vimeo-90k](#id6)
    
    
5. <a id="index"></a>[PASOS PREVIOS AL TRAIN](#id7)    


6. <a id="index"></a>[PASOS PREVIOS AL TEST](#id8)


7. <a id="index"></a>[TRAIN AND TEST](#id9)

    7.1. <a id="index"></a>[TRAIN 1 GPU](#id10)
        
    7.2. <a id="index"></a>[TEST 1 GPU](#id11)
    
    7.3. <a id="index"></a>[TEST 8 GPU](#id12)
    
    7.4. <a id="index"></a>[TEST 1 GPU Slurm](#id13)
    
    
8. <a id="index"></a>[EJECUCIÓN](#id14)    


9. <a id="index"></a>[DEMOSTRACIÓN](#id15)



#  ¿NTIRE19 CHALLENGE?<a name="id1"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

New Trends in Image Restoration and Enhancement workshop and challenges on image and video restoration and enhancement.
Proponen diversos retos en las áreas de la restauración de imágenes y vídeos y los participantes compiten para obtener los mejores resultados.

EDVR es la solución ganadora del NTIRE19 Challenge en la categoría de super resolución de vídeos, centrada en dos aspectos:

* Alienación de múltiples frames con mucho movimiento.
* Fusión de frames con distintos movimientos y partes borrosas. 

EDVR ganó en las 4 pruebas relacionadas con vídeos con amplio margen. 

#  ¿QUÉ ES EDVR?<a name="id2"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

EDVR es el acrónimo de Enhanced Deformable convolutions Video Restoration, es una herramienta contenida en BasicSR, centrada en la super resolución y desemborronamiento de vídeos.

Está basada en dos principios el PCD (Pyramid,
Cascading and Deformable convolutions) y el TSA (Temporal and Spatial Attention)

* PCD: usa convoluciones deformables para agrupar píxeles cercanos, usando una estructura piramidal invertida empezando a pequeña escala y progresivamente aumentando, junto a esto se aplica un efecto cascada para mejorar resultados. 
* TSA: es un módulo de fusión que agrega información comparando frames con sus anteriores y posteriores, toda esa información luego se usa en todos los frames.


<img src="ejemplos/FrameworkEDVR.PNG">

### Estructura de EDVR

# PASOS PREVIOS<a name="id3"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

Todos estos pasos pueden seguirse para descargar y poder usar el [repositorio original de EDVR](https://github.com/xinntao/EDVR). Si no se quieren descargar todos los datasets puede descargar mi repositorio en dentro del original y comprobar los resultados. En el apartado de [EJECUCIÓN](#id14) se exponen las dos opciones a la hora de realizar la prueba.

Para poder ejecutar EDVR es necesario cumplir los siguientes requisitos:

+ Tener Python => 3.7

+ PyTorch >= 1.3

+ NVIDIA GPU compatible con CUDA
 
+ Clonar el repositorio

Una vez cumplidos ejecutamos estas dos casillas y comprobará que cumple los requisitos e instalaran los archivos que falten.

In [2]:
 !pip install -r ../requirements.txt

In [2]:
 !python ../setup.py develop

# DATASETS<a name="id4"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

Para las pruebas se han usado 2 datasets, REDS dataset y Vimeo-90k dataset. Debido a que ambos datasets ocupan mucho, no están en el repositorio, pero aquí están los pasos para descargarlos y poder ejecutar EDVR.

## REDS<a name="id5"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
Es el dataset oficial del NTIRE 2019 Y NTIRE 2020 Challenge, acrónimo de REalistic and Dynamic Scenes está centrado en el desemborronado y super resolución de vídeos. Tiene diferentes apartados con diferentes situaciones, imágenes borrosas, borrosas y comprimidas, en baja resolución y borrosas y en baja resolución.

```bash
@InProceedings{Nah_2019_CVPR_Workshops_REDS,
  author = {Nah, Seungjun and Baik, Sungyong and Hong, Seokil and Moon, Gyeongsik and Son, Sanghyun and Timofte, Radu and Lee, Kyoung Mu},
  title = {NTIRE 2019 Challenge on Video Deblurring and Super-Resolution: Dataset and Study},
  booktitle = {The IEEE Conference on Computer Vision and Pattern Recognition (CVPR) Workshops},
  month = {June},
  year = {2019}
}
```
REDS se puede obtener [aquí](https://seungjunnah.github.io/Datasets/reds.html). 
Aquí se nos explica brevemente el dataset y se nos ofrecen dos tipos de descarga, mediante Google Drive y SNU CVLab server. Una vez descomprimido el dataset ocupa 291 GB

La opción que he usado es Google Drive, a veces Google bloquea el acceso a los ficheros, con algo de paciencia se consiguen todos, la otra opción no conseguí que funcionase en mi caso. También en esta web hay un script de Python, pero  por lo expuesto, tampoco conseguí que descargase las carpetas comprimidas del dataset.

Una vez descargadas todas las carpetas hay que extraer las imágenes, el dataset contiene las imágenes en baja resolución y las originales. Los path pueden cambiar y habría que modificar los archivos  .yml contenidos en la carpeta options, en nuestro caso tanto los contenidos en train/EDVR y test/EDVR referentes a este dataset, más adelante se explicará el contenido de estos archivos con un ejemplo.

Una vez hecho esto ejecutamos un scipt de Python para comprobar que todo está bien, solo habrá que modificar en el script los path al dataset. /tests/test_reds_dataset.py

Con esto ya tendríamos REDS preparado para trabajar.


## Vimeo-90k<a name="id6"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
Dataset creado original mente para el proyecto Video Enhancement with Task-Oriented Flow, consistente en 89.800 clips de vídeo procedentes de vimeo.com, una de sus principales funciones entre otra es servir para la super resolución de vídeos. La versión que se utiliza consta de 91.701 secuencias de 7 frames con una resolución fija.


La cita pertenece a el trabajo para el que se creó el dataset, no sé si debería mencionarlo o no?¿

```
 @article{xue2019video,
  title={Video Enhancement with Task-Oriented Flow},
  author={Xue, Tianfan and Chen, Baian and Wu, Jiajun and Wei, Donglai and Freeman, William T},
  journal={International Journal of Computer Vision (IJCV)},
  volume={127},
  number={8},
  pages={1106--1125},
  year={2019},
  publisher={Springer}
}
```
Vimeo-90k se puede obtener [aquí](http://toflow.csail.mit.edu/).

Al final de la web se nos presentan diferentes versiones del dataset, en mi caso se usa el último, The original training + test set (82GB). Una vez descomprimido y con las imágenes en baja resolución generadas ocupa 184 GB.

Una vez descargado y extraído obtenemos las imágenes en alta resolución solamente, ahora tenemos que crear las imágenes en baja resolución. Para ello usaremos un script de Matlab. Este script se encuentra dentro de la carpeta /scripts/matlab_scripts y con el nombre de generate_LR_Vimeo90K.m

Es posible que se tenga que modificar el path dentro del script.

Una vez creadas las imágenes en baja resolución igual que en REDS hay un script de Python para comprobar que todo está bien.
Situado en /tests/test_vimeo90k_dataset.py

Con esto ya tendríamos Vimeo-90k preparado para trabajar.


# PASOS PREVIOS AL TRAIN <a name="id7"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>


```yaml
name: 102_EDVR_M_x4_SR_REDS_600k_B4G8_valREDS4_101pretrain_wandb
# es el nombre de la ejecución, que puede cambiarse 
model_type: EDVRModel
# el modelo siempre será el mismo ya que solo trabajamos con EDVR en BasicSR
scale: 4
# es la escala de la salida respecto a la entrada, el upsampling ratio.
num_gpu: 8 
# en todas mis ejecuciones el número de Gpu es una por mis limitaciones hardware, que se indica por parámetro, con 0 se usa la Cpu
manual_seed: 10
```

```yaml
datasets:
  train:
    name: REDS
    # el nombre del dataset
    type: REDSDataset
    # es el tipo de dataset que se usa:
    # 1. subfolder (clip) name; 2. frame number; 3. image shape
    #  000 100 (720,1280,3)
    dataroot_gt: datasets/REDS/train_sharp/train_sharp
    # path a las imágenes originales
    dataroot_lq: datasets/REDS/train_sharp_bicubic/train/train_sharp_bicubic/X4
    # path a las imágenes en baja calidad
    dataroot_flow: ~
    meta_info_file: basicsr/data/meta_info/meta_info_REDS_GT.txt
    # selección de sub-carpetas con las que se ejecutara EDVR
    val_partition: REDS4  
   
    io_backend:
      type: disk
    # hay tres opciones para la ejecución disk, LMDB o memcached, solo se ha usado disk para los experimentos
    
    num_frame: 5
    # número de frames
    gt_size: 256
    # tamaño del Ground-Truth 
    interval_list: [1]
    random_reverse: false
    use_flip: true
    # usa la rotación de 180 grados
    use_rot: true
    # usa la rotación de 90 grados 
    
    use_shuffle: true
    # se baraja 
    num_worker_per_gpu: 3
    # número de lecturas de cada GPU
    batch_size_per_gpu: 4
    # tamaño total de la ejecución
    dataset_enlarge_ratio: 200
    # el ratio de ampliación del dataset 
    prefetch_mode: ~

  val:
    name: REDS4
    # nombre del dataset de validación
    type: VideoTestDataset
    # es el tipo de dataset que se usa:
    # 1. clip name; 2. frame number; 3. number of frames; 4. tamaño de la imagen 
    #  00001/0001 7 (256,448,3)
    dataroot_gt: datasets/REDS/train_sharp
    # path a las imágenes originales
    dataroot_lq: datasets/REDS/train_sharp_bicubic
    # path a las imágenes en baja calidad
    meta_info_file: basicsr/data/meta_info/meta_info_REDS4_test_GT.txt
    # selección de sub-carpetas con las que se ejecutara EDVR
    io_backend:
      type: disk
    # hay tres opciones para la ejecución disk, LMDB o memcached, solo se ha usado disk para los experimentos
    
    cache_data: false
    num_frame: 5
    # número de 
    padding: reflection_circle
    # replicate o reflection_circle, son dos técnicas para conseguir que durante el procesamiento el tamaño de las 
    # imágenes no cambien.
```

```yaml
network_g:
  type: EDVR
  # tipo de arquitectura que se usa, solo usaremos EDVR
  num_in_ch: 3
  # número de canales de entradas de inputs
  num_out_ch: 3
  # número de canales de salidas de inputs 
  num_feat: 64
  # número de canales intermedios 
  num_frame: 5
  # número de frames
  deformable_groups: 8
  num_extract_block: 5
  num_reconstruct_block: 10
  center_frame_idx: ~
  hr_in: false
  with_predeblur: false
  # tenemos la opción de activar o desactivar la opción de desemborronado previo
  with_tsa: true
  # tenemos la opción de activar o desactivar la opción de TSA de fusión de frames
  ```

```yaml
path:
  pretrain_network_g: experiments/101_EDVR_M_x4_SR_REDS_woTSA_600k_B4G8_valREDS4_wandb/models/net_g_600000.pth
  # path al modelo pre-entrenado
  strict_load_g: false
  # si solo queremos cargar el modelo pre-entrenado
  resume_state: ~
  # si una ejecución  se interrumpe, podemos continuar
```


```yaml
train:
  optim_g:
    type: Adam
    # tipo de optimizador
    lr: !!float 4e-4
    # ratio de aprendizaje 
    weight_decay: 0
    betas: [0.9, 0.99]
    # beta1 and beta2 para Adam

  scheduler:
    type: CosineAnnealingRestartLR
    # tipo de programador
    periods: [50000, 100000, 150000, 150000, 150000]
    # periodos de alineamiento de coseno 
    restart_weights: [1, 1, 1, 1, 1]
    # pesos de reinicio del alineamiento de coseno 
    eta_min: !!float 1e-7
    # tasa mínima de aprendizaje   

  total_iter: 600000
  # número de interacciones total 
  warmup_iter: -1 
  # pruebas de calentamiento, -1 es ninguna
  tsa_iter: 50000
  # número de interacciones en modo tsa
  dcn_lr_mul: 1

  pixel_opt:
    type: CharbonnierLoss
    # tipo de pérdida 
    loss_weight: 1.0
    # peso de la pérdida
    reduction: sum
    # modo de reducción
```

```yaml
val:
  val_freq: !!float 5e3
  # frecuencia de validación 
  save_img: false
  # si se guardan las imágenes durante la validación

  metrics:
  # métricas extra que calculan el PSNR(Peak Signal-to-Noise Ratio)
    psnr: 
      type: calculate_psnr
      crop_border: 0
      test_y_channel: false
```

```yaml
logger:
  print_freq: 100
  # frecuencia de logger 
  save_checkpoint_freq: !!float 5e3
  # la frecuencia para guardar puntos de control
  use_tb_logger: true
  # si se usa tensorboard logger
  wandb:
  # si se usa wandb logger
    project: ~
    resume_id: ~
```

``` yaml
dist_params:
# solo se usa con slurm training 
  backend: nccl
  port: 29500

find_unused_parameters: true

```

# PASOS PREVIOS AL TEST<a name="id8"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

Lo primero es seleccionar los datasets o imágenes con las que se va a trabajar, si se quisiese usar un vídeo primero habría que generar la secuencia de frames y las imágenes en baja calidad.

Una vez hecho esto hay que crear un fichero .yml con las siguientes especificaciones:

```yaml
name: EDVR_L_x4_REDS_SRblur_official
# es el nombre de la ejecución, que puede cambiarse 
model_type: EDVRModel
# el modelo siempre será el mismo ya que solo trabajamos con EDVR en BasicSR
scale: 4
# es la escala de la salida respecto a la entrada, el upsampling ratio.
num_gpu: 4
# en todas mis ejecuciones el número de Gpu es una por mis limitaciones hardware, que se indica por parámetro, on 0 se usa la cpu
manual_seed: 0
```

```yaml
datasets:
  test:
    name: REDS4
    # el nombre del dataset
    type: VideoTestDataset
    # es el tipo de dataset que se usa, depende de la estructura de los dataset, en nuestro caso se usan dos:
    # video_test_dataset: 
    #                      dataroot
    #                      ├── subfolder1
    #                           ├── frame000
    #                           ├── frame001
    #                           ├── ...
    #                      ├── subfolder1
    #                           ├── frame000
    #                           ├── frame001
    #                           ├── ...
    #                       ├── ...
    #                       
    # vimeo90k_dataset:  1. clip name; 2. frame number; 3. number of frames; 4. tamaño de la imágen 
    #  00001/0001 7 (256,448,3)
    dataroot_gt: datasets/REDS/train_sharp/train/train_sharp
    # path a las imágenes originales
    dataroot_lq: datasets/REDS/train_blur_bicubic/train/train_blur_bicubic/X4
    # path a las imágenes en baja calidad
    meta_info_file: basicsr/data/meta_info/meta_info_REDS4_test_GT.txt
    # selección de sub-carpetas con las que se ejecutara EDVR
    io_backend:
      type: disk
    # hay tres opciones para la ejecución disk, LMDB o memcached, solo se ha usado disk para los experimentos
    cache_data: false
    num_frame: 5
    # número de frames
    padding: replicate
    # replicate o reflection_circle, son dos técnicas para conseguir que durante el procesamiento el tamaño de las 
    # imagenes no cambien.
```   

```yaml
network_g:
  type: EDVR
  # tipo de arquitectura que se usa, solo usaremos EDVR
  num_in_ch: 3
  # número de canales de entradas de inputs 
  num_out_ch: 3
  # número de canales de salidas de inputs 
  num_feat: 128
  # número de canales intermedios 
  num_frame: 5
  # número de frames
  deformable_groups: 8
  num_extract_block: 5
  num_reconstruct_block: 40
  center_frame_idx: ~
  hr_in: false
  with_predeblur: true
  # tenemos la opción de activar o desactivar la opción de desemborronado previo
  with_tsa: true
  # tenemos la opción de activar o desactivar la opción de TSA de fusión de frames
```


```yaml
path:
  pretrain_network_g: experiments/pretrained_models/EDVR/EDVR_L_x4_SRblur_REDS_official-983d7b8e.pth
  # path al modelo pre-entrenado
  strict_load_g: true
  # si solo queremos cargar el modelo pre-entrenado
```

```yaml
val:
  save_img: true
  # decide si se guardan las imágenes durante la validación
  suffix: ~  
```

 ```yaml
 metrics:
 # métricas extra que calculan el PSNR(Peak Signal-to-Noise Ratio)
    psnr: y
      type: calculate_psnr
      crop_border: 0
      test_y_channel: false
```

# TRAIN AND TEST<a name="id9"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

Ambas opciones tienen varias opciones de ejecución, aunque en mi caso solo se ha experimentado con una debida a las limitaciones hardware.
La versión que se ha usado es la de una sola GPU, también se puede trabajar con múltiples GPU(4 y 8), y por último  se puede trabajar en modo Slurm con 1, 4 o 8 GPU.

La forma de ejecutar el script es muy parecida, pero para las versiones multiGPU hay que usar más parámetros opcionales.

La única diferencia en este apartado respecto al el modo train o test se corresponde al path de archivos .yml que analizamos anteriormente.

## TRAIN 1 GPU<a name="id10"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

```bash
PYTHONPATH="./:${PYTHONPATH}" \
CUDA_VISIBLE_DEVICES=0 \
python basicsr/train.py -opt options/train/EDVR/train_EDVR_L_x4_SR_REDS.yml
```

## TEST 1 GPU<a name="id11"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

```bash
PYTHONPATH="./:${PYTHONPATH}" \
CUDA_VISIBLE_DEVICES=0 \
python basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SRblur_REDS.yml
```

## TEST 8 GPU<a name="id12"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

```bash
PYTHONPATH="./:${PYTHONPATH}" \
CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 \
python -m torch.distributed.launch --nproc_per_node=8 --master_port=4321 basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SRblur_REDS.yml --launcher pytorch
```

## TEST 1 GPU Slurm<a name="id13"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>
```bash
PYTHONPATH="./:${PYTHONPATH}" \
GLOG_vmodule=MemcachedClient=-1 \
srun -p [partition] --mpi=pmi2 --job-name=test --gres=gpu:1 --ntasks=1 --ntasks-per-node=1 --cpus-per-task=6 --kill-on-bad-exit=1 \
python -u basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SRblur_REDS.yml --launcher="slurm"
```

# EJECUCIÓN<a name="id14"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>


A continuación se ejecutará una de las versiones del dataset REDS.
Si se ha seguido los pasos descritos anteriormente y se tienen los dataset descargados correctamente use las líneas que vienen por defecto.
Si no, elimine el # en la línea comentada y comente la que no estaba comentada, así usara los recursos proporcionados en mi repositorio.

In [1]:
!PYTHONPATH="./:${PYTHONPATH}"

In [2]:
!CUDA_VISIBLE_DEVICES=0

In [4]:
#!python ../basicsr/test.py -opt miniConjunto/test_EDVR_L_x4_SRblur_REDS.yml

In [3]:
!python ../basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SRblur_REDS.yml

Disable distributed.
Path already exists. Rename it to /home/edvr/edvr/BasicSR/results/EDVR_L_x4_REDS_SRblur_official_archived_20210223_231639
2021-02-23 23:16:39,255 INFO: 
                ____                _       _____  ____
               / __ ) ____ _ _____ (_)_____/ ___/ / __ \
              / __  |/ __ `// ___// // ___/\__ \ / /_/ /
             / /_/ // /_/ /(__  )/ // /__ ___/ // _, _/
            /_____/ \__,_//____//_/ \___//____//_/ |_|
     ______                   __   __                 __      __
    / ____/____   ____   ____/ /  / /   __  __ _____ / /__   / /
   / / __ / __ \ / __ \ / __  /  / /   / / / // ___// //_/  / /
  / /_/ // /_/ // /_/ // /_/ /  / /___/ /_/ // /__ / /<    /_/
  \____/ \____/ \____/ \____/  /_____/\____/ \___//_/|_|  (_)
    
Version Information: 
	BasicSR: 1.2.0+1464d8e
	PyTorch: 1.7.0
	TorchVision: 0.8.1
2021-02-23 23:16:39,255 INFO: 
  name: EDVR_L_x4_REDS_SRblur_official
  model_type: EDVRModel
  scale: 4
  num_gpu: 4
  manual_seed: 0
  da

2021-02-23 23:16:45,810 INFO: Model [EDVRModel] is created.
2021-02-23 23:16:45,810 INFO: Testing REDS4...
2021-02-23 23:16:45,810 WARNING: nondist_validation is not implemented. Run dist_validation.
Test 020:100/100: 100%|████████████████████| 400/400 [03:32<00:00,  1.88frame/s]
2021-02-23 23:20:18,483 INFO: Validation REDS4
	 # psnr: 28.8819	 # 000: 27.3231	 # 011: 29.4919	 # 015: 31.2684	 # 020: 27.4444



# DEMOSTRACIÓN<a name="id15"></a><a href="#index"><i class="fa fa-list-alt" aria-hidden="true"></i></a>

<img src="ejemplos/00000004.png">

### Imagen original en baja calidad de REDS

<img src="ejemplos/00000004_EDVR_L_x4_REDS_SRblur_official.png">

### Imagen procesada por EDVR

<img src="ejemplos/00000009.png">

### Imagen original en baja clidad de REDS

<img src="ejemplos/00000009_EDVR_L_x4_REDS_SRblur_official.png">

### Imagen procesada por EDVR

PYTHONPATH="./:${PYTHONPATH}"
CUDA_VISIBLE_DEVICES=0
python basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SR_Mio.yml

PYTHONPATH="./:${PYTHONPATH}"
CUDA_VISIBLE_DEVICES=0
python basicsr/test.py -opt options/test/EDVR/test_EDVR_L_x4_SR_NO_INTERFAZ.yml